In [1]:
using JuMP, GLPK, Distributed

module BranchAndBoundSystem

using JuMP, GLPK

export Node, solveRelaxedProblem, computeBounds, branch_and_bound, SelectVarMaxRange, local_OPT, global_OPT_base

mutable struct Node
    model::Model
    lowerBound::Float64
    upperBound::Float64
    level::Int
    parent::Union{Nothing, Node}

    function Node(model::Model, lowerBound::Float64 = -Inf, upperBound::Float64 = Inf, level::Int = 0, parent::Union{Nothing, Node} = nothing)
        new(model, lowerBound, upperBound, level, parent)
    end
end

function solveRelaxedProblem(node::Node)
    optimize!(node.model)
    status = termination_status(node.model)
    if status == MOI.OPTIMAL
        return objective_value(node.model), [value(v) for v in all_variables(node.model)]
    else
        return -Inf, []
    end
end

function validate_objective(node::Node)
    x1_val = value(node.model[:x1])
    x2_val = value(node.model[:x2])
    calculated_obj = 100 * x1_val + 150 * x2_val
    reported_obj = objective_value(node.model)
    println("Manual Obj: $calculated_obj, Reported Obj: $reported_obj")
    if abs(calculated_obj - reported_obj) > 1e-6
        error("Objective function mismatch detected")
    end
end

function computeBounds(node::Node)
    obj_value, solution = solveRelaxedProblem(node)
    validate_objective(node)  # Ensure objective validation
    println("Node Level: $(node.level), Obj Value: $obj_value, Current Lower Bound: $(node.lowerBound)")
    if obj_value > node.lowerBound
        node.lowerBound = obj_value
        println("Updated Lower Bound: $(node.lowerBound)")
    end
end

function getname(var::VariableRef)
    return Symbol(name(var))
end

function branch!(node::Node)
    selected_var = SelectVarMaxRange(node)
    if selected_var !== nothing
        value_var = value(selected_var)
        floor_val = floor(value_var)
        ceil_val = ceil(value_var)
        println("Branching on var: $(getname(selected_var)), Value: $value_var, Floor: $floor_val, Ceil: $ceil_val")
        if floor_val != ceil_val
            model1 = copy(node.model)
            set_optimizer(model1, GLPK.Optimizer)
            var1 = model1[getname(selected_var)]
            @constraint(model1, var1 <= floor_val)
            node1 = Node(model1, node.lowerBound, node.upperBound, node.level + 1, node)

            model2 = copy(node.model)
            set_optimizer(model2, GLPK.Optimizer)
            var2 = model2[getname(selected_var)]
            @constraint(model2, var2 >= ceil_val)
            node2 = Node(model2, node.lowerBound, node.upperBound, node.level + 1, node)

            println("Created two new nodes with bounds: $(node1.lowerBound), $(node2.upperBound)")
            return [node1, node2]
        end
    end
    println("No branching occurred for node at level $(node.level)")
    return []
end

function SelectVarMaxRange(node::Node)
    max_range = 0.0
    selected_var = nothing
    println("Selecting variable to branch on:")
    for v in all_variables(node.model)
        var_upper_bound = has_upper_bound(v) ? upper_bound(v) : Inf
        var_lower_bound = has_lower_bound(v) ? lower_bound(v) : -Inf
        var_range = var_upper_bound - var_lower_bound
        println("Variable $(getname(v)), Range: $var_range")
        if var_range > max_range
            max_range = var_range
            selected_var = v
        end
    end
    println("Selected Variable: $(getname(selected_var)), Max Range: $max_range")
    return selected_var
end

function branch_and_bound(root::Node)
    active_nodes = [root]
    best_solution = nothing
    best_cost = -Inf

    while !isempty(active_nodes)
        current_node = popfirst!(active_nodes)
        println("Processing Node at Level: $(current_node.level), Lower Bound: $(current_node.lowerBound)")
        computeBounds(current_node)

        if current_node.lowerBound > best_cost
            if is_solution_integer(current_node) && objective_value(current_node.model) > best_cost
                println("Found new potential solution with cost: $(objective_value(current_node.model))")
                best_solution = current_node
                best_cost = objective_value(current_node.model)
            end
            # Always check if the node still has potential to contain a better solution
            if objective_value(current_node.model) > best_cost
                new_nodes = branch!(current_node)
                foreach(n -> println("Generated child node with bounds: Lower = $(n.lowerBound), Upper = $(n.upperBound)"), new_nodes)
                append!(active_nodes, new_nodes)
            else
                println("Pruning Node at Level: $(current_node.level) due to non-improving bound $(current_node.lowerBound) <= $(best_cost)")
            end
        end
    end

    if best_solution !== nothing
        println("Best cost found: ", best_cost)
        println("Best solution found: x1 = ", value(best_solution.model[:x1]), ", x2 = ", value(best_solution.model[:x2]))
    else
        println("No valid solution was found.")
    end

    return best_solution, best_cost
end



function is_solution_integer(node::Node)
    return all(is_integer(value(v)) for v in all_variables(node.model))
end

function is_integer(x::Float64)
    return abs(x - round(x)) < 1e-6
end

function init_bounds(model::Model, default_lower=-Inf, default_upper=Inf)
    for var in all_variables(model)
        set_lower_bound(var, default_lower)
        set_upper_bound(var, default_upper)
    end
end

function local_OPT(model::Model)
    set_optimizer(model, GLPK.Optimizer)
    optimize!(model)
    return objective_value(model), [value(var) for var in all_variables(model)]
end

function global_OPT_base(model::Model)
    set_optimizer(model, GLPK.Optimizer)
    optimize!(model)
    if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.FEASIBLE
        return objective_value(model), [value(var) for var in all_variables(model)]
    else
        return Inf, []
    end
end

end  # End of module BranchAndBoundSystem

using .BranchAndBoundSystem

function create_example_model()
    model = Model(GLPK.Optimizer)
    @variable(model, x1 >= 0)
    @variable(model, x2 >= 0)
    @objective(model, Max, 100x1 + 150x2)
    @constraint(model, constraint1, 8000x1 + 4000x2 <= 40000)
    @constraint(model, constraint2, 15x1 + 30x2 <= 200)
    return model
end

function run_branch_and_bound()
    model = create_example_model()
    root_node = Node(model)
    best_solution, best_cost = branch_and_bound(root_node)
    
    if best_solution !== nothing
        x1_val = value(best_solution.model[:x1])
        x2_val = value(best_solution.model[:x2])
        println("Best cost found: ", best_cost)
        println("Best solution found: x1 = ", x1_val, ", x2 = ", x2_val)
    else
        println("No valid solution was found.")
    end

    return best_solution, best_cost
end

t_bb = @elapsed best_solution, best_cost = run_branch_and_bound()
println("Best cost found: ", best_cost)
println("Best solution found: ", best_solution)


Processing Node at Level: 0, Lower Bound: -Inf
Manual Obj: 1055.5555555555557, Reported Obj: 1055.5555555555557
Node Level: 0, Obj Value: 1055.5555555555557, Current Lower Bound: -Inf
Updated Lower Bound: 1055.5555555555557
Selecting variable to branch on:
Variable x1, Range: Inf
Variable x2, Range: Inf
Selected Variable: x1, Max Range: Inf
Branching on var: x1, Value: 2.222222222222222, Floor: 2.0, Ceil: 3.0
Created two new nodes with bounds: 1055.5555555555557, Inf
Generated child node with bounds: Lower = 1055.5555555555557, Upper = Inf
Generated child node with bounds: Lower = 1055.5555555555557, Upper = Inf
Processing Node at Level: 1, Lower Bound: 1055.5555555555557
Manual Obj: 1050.0, Reported Obj: 1050.0
Node Level: 1, Obj Value: 1050.0, Current Lower Bound: 1055.5555555555557
Selecting variable to branch on:
Variable x1, Range: Inf
Variable x2, Range: Inf
Selected Variable: x1, Max Range: Inf
Branching on var: x1, Value: 2.0, Floor: 2.0, Ceil: 2.0
No branching occurred for nod